<a href="https://colab.research.google.com/github/kavyajeetbora/ETL_wages/blob/master/WIOD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation and Import

In [ ]:
# !pip install -q pyxlsb

In [2]:
%%time
# !wget https://dataverse.nl/api/access/datafile/199104 -O wiot.zip
!wget https://dataverse.nl/api/access/datafile/199097 -O exchange_rates.xlsx
!wget https://dataverse.nl/api/access/datafile/199095 -O sea.xlsx
!wget https://dataverse.nl/api/access/datafile/354108 -O NA_data.xlsx
!wget https://raw.githubusercontent.com/kavyajeetbora/ETL_wages/master/data/PPP_export.csv -O ppp.csv

--2024-07-22 15:58:48--  https://dataverse.nl/api/access/datafile/199097
Resolving dataverse.nl (dataverse.nl)... 194.171.144.89
Connecting to dataverse.nl (dataverse.nl)|194.171.144.89|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19308 (19K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘exchange_rates.xlsx’

exchange_rates.xlsx 100%[===================>]  18.86K  --.-KB/s    in 0.1s    

2024-07-22 15:58:49 (144 KB/s) - ‘exchange_rates.xlsx’ saved [19308/19308]

--2024-07-22 15:58:49--  https://dataverse.nl/api/access/datafile/199095
Resolving dataverse.nl (dataverse.nl)... 194.171.144.89
Connecting to dataverse.nl (dataverse.nl)|194.171.144.89|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5536437 (5.3M) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘sea.xlsx’

sea.xlsx            100%[===================>]   5.28M  4.89MB/s    in 1.1s    

2024-07-22 15:58:51 (4

In [3]:
import pandas as pd
from zipfile import ZipFile
from tqdm import tqdm
# from pyxlsb import open_workbook as open_xlsb

## SEA

In [4]:
%%time
sea = pd.read_excel("sea.xlsx", sheet_name='DATA')

CPU times: user 10.7 s, sys: 114 ms, total: 10.8 s
Wall time: 11 s


In [5]:
sea.head(3)

,country,variable,description,code,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,AUS,CAP,"Crop and animal production, hunting and relate...",A01,9739.77,13729.03,9726.30,11808.62,11163.98,12429.90,8706.66,11227.29,12556.71,13768.79,18441.27,20106.04,19659.28,18986.06,20525.22
1,AUS,CAP,Forestry and logging,A02,582.69,740.04,540.79,651.96,649.34,676.64,549.27,616.70,716.86,749.91,894.94,923.40,1001.44,1247.96,1206.44
2,AUS,CAP,Fishing and aquaculture,A03,869.31,1122.29,652.69,1024.72,1059.83,923.46,904.15,863.98,1146.37,1108.94,1116.14,1213.94,1182.11,1388.98,1475.22


In [121]:
year_range = list(range(2000,2015))

## Convert to USD $

In [51]:
exchange = pd.read_excel('exchange_rates.xlsx', sheet_name='EXR', header=3)
exchange.columns = list(exchange.columns[:2].str.lower()) + [int(c.replace("_", "")) for c in exchange.columns[2:]]
exchange = pd.melt(exchange, id_vars=['acronym'], value_vars=list(range(2000,2015)), var_name='Year')
exchange.head(3)

,acronym,Year,value
0,AUS,2000,0.582346
1,AUT,2000,0.923600
2,BEL,2000,0.923600


In [53]:
SEA = pd.melt(sea, id_vars=['country','variable','code'], value_vars= year_range, var_name='year')
SEA.head(3)

,country,variable,code,year,value
0,AUS,CAP,A01,2000,9739.77
1,AUS,CAP,A02,2000,582.69
2,AUS,CAP,A03,2000,869.31


In [75]:
SEA_USD = pd.merge(left=SEA,
         right=exchange,
         how='left',
         left_on=SEA['country']+SEA['year'].astype(str),
         right_on=exchange['acronym']+exchange['Year'].astype(str),
         suffixes=('', '_y')
)

SEA_USD['value'] = SEA_USD['value']*SEA_USD['value_y']
SEA_USD = SEA_USD.drop(['key_0', 'value_y', 'acronym', 'Year'], axis=1)

## Unpivot changes
SEA_USD = SEA_USD.pivot(columns=['year'], index=['country','variable', 'code'], values='value').reset_index()
SEA_USD.head(3)

year,country,variable,code,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,AUS,CAP,A01,5671.916100,7106.640173,5290.163749,7697.850440,8223.153224,9494.504536,6559.162311,9414.767530,10739.314578,10895.298602,16956.729324,20753.655548,20363.082224,18386.613128,18528.013468
1,AUS,CAP,A02,339.327191,383.071345,294.137303,425.002293,478.290208,516.847404,413.792554,517.140569,613.105268,593.406783,822.896435,953.142714,1037.291552,1208.558159,1089.047356
2,AUS,CAP,A03,506.239201,580.937706,355.000049,667.998572,780.648522,705.379380,681.141402,724.499933,980.450138,877.508658,1026.289614,1253.041007,1224.429538,1345.125734,1331.673718


## Adjusted USD $

In [80]:
GO_PI = sea.loc[sea['variable']=='GO_PI'].copy()
GO_PI[GO_PI.columns[4:]] = GO_PI[GO_PI.columns[4:]]/100
GO_PI = pd.melt(GO_PI, id_vars=['country','variable', 'code'], value_vars=year_range, var_name='Year')
GO_PI.head(3)

,country,variable,code,Year,value
0,AUS,GO_PI,A01,2000,0.91576
1,AUS,GO_PI,A02,2000,0.91576
2,AUS,GO_PI,A03,2000,0.91576


In [95]:
SEA = SEA_USD.copy()
SEA = pd.melt(SEA, id_vars=['country','variable','code'], value_vars= year_range, var_name='year')
SEA.head(3)

,country,variable,code,year,value
0,AUS,CAP,A01,2000,5671.916100
1,AUS,CAP,A02,2000,339.327191
2,AUS,CAP,A03,2000,506.239201


In [97]:
adj_USD = pd.merge(left=SEA,
         right=GO_PI,
         how='left',
         left_on=SEA['country']+SEA['year'].astype(str)+SEA['code'],
         right_on=GO_PI['country']+GO_PI['Year'].astype(str)+GO_PI['code'],
         suffixes=('', '_y')
)
adj_USD['value'] = adj_USD['value']*adj_USD['value_y']
adj_USD = adj_USD.drop(['key_0', 'country_y','variable_y','code_y', 'Year', 'value_y'], axis=1)

## Unpivot changes
adj_USD = adj_USD.pivot(columns=['year'], index=['country','variable', 'code'], values='value').reset_index()
adj_USD.sample(4)

year,country,variable,code,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
7977,CYP,VA_PI,E37-E39,70.383206,69.882383,76.101827,101.611624,100.131247,78.894948,119.055502,132.589724,168.653237,128.544768,132.57000,156.744674,204.408930,197.988864,177.581385
18871,IND,CAP,U,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
9741,DEU,VA,R_S,60354.368024,61151.840334,67230.765684,83655.148519,95674.531183,97342.753677,101542.264390,114036.811786,128931.247544,121422.375033,119379.28500,130433.901966,124432.617233,136854.152739,145322.647120
29772,POL,EMPE,J58,7.677761,7.511644,7.575530,7.873337,9.130035,10.676383,12.319046,15.069052,17.709494,12.858598,14.96493,14.848874,11.331182,13.078861,13.643611


## Industry PPP in Adjusted USD $



In [118]:
ppp1 = adj_USD.groupby(['code','variable']).sum().reset_index()
ppp1 = ppp1.drop(['country'], axis=1)
ppp1[ppp1['variable']=='GO'].sample(5)

year,code,variable,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
628,J62_J63,GO,7.103933e+05,7.434941e+05,7.752886e+05,8.528993e+05,9.492604e+05,1.019677e+06,1.108827e+06,1.267954e+06,1.450451e+06,1.412496e+06,1.524283e+06,1.742641e+06,1.808271e+06,1.906810e+06,2.002076e+06
804,O84,GO,2.791971e+06,2.930259e+06,3.192383e+06,3.665721e+06,4.152281e+06,4.574809e+06,5.023863e+06,5.678521e+06,6.544449e+06,6.748054e+06,7.279122e+06,8.000631e+06,8.253982e+06,8.535141e+06,8.912744e+06
244,C25,GO,7.962155e+05,7.529717e+05,7.550810e+05,8.469596e+05,1.024436e+06,1.165736e+06,1.336022e+06,1.623025e+06,1.902867e+06,1.594108e+06,1.738844e+06,2.064517e+06,2.155747e+06,2.241908e+06,2.293381e+06
212,C23,GO,4.423309e+05,4.205225e+05,4.251402e+05,4.845866e+05,5.626094e+05,6.450286e+05,7.685008e+05,9.533902e+05,1.104176e+06,1.054767e+06,1.127425e+06,1.393049e+06,1.456481e+06,1.635760e+06,1.670225e+06
52,B,GO,3.487707e+05,3.494163e+05,3.366192e+05,4.600620e+05,6.576114e+05,1.088014e+06,1.394192e+06,1.667507e+06,2.632544e+06,1.662409e+06,2.464841e+06,3.530177e+06,3.530350e+06,3.578666e+06,3.651147e+06


## Industry Total in Current Dollars

In [119]:
ppp2 = SEA_USD.groupby(['code','variable']).sum().reset_index()
ppp2 = ppp2.drop(['country'], axis=1)
ppp2[ppp2['variable']=='GO'].sample(5)

year,code,variable,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
596,J59_J60,GO,3.959748e+05,3.915468e+05,4.155504e+05,4.598358e+05,5.003156e+05,5.269995e+05,5.525683e+05,5.921739e+05,6.171743e+05,5.887252e+05,6.276169e+05,6.674842e+05,6.750082e+05,6.843355e+05,6.968972e+05
500,H50,GO,2.191204e+05,2.224647e+05,2.330753e+05,2.711102e+05,3.237241e+05,3.617090e+05,3.834806e+05,4.562852e+05,5.155289e+05,4.158260e+05,4.584756e+05,5.079138e+05,5.188425e+05,5.251842e+05,5.451889e+05
756,M73,GO,3.446236e+05,3.357065e+05,3.353101e+05,3.745959e+05,4.215442e+05,4.465187e+05,4.694549e+05,5.163452e+05,5.516610e+05,4.961110e+05,5.166785e+05,5.471471e+05,5.447251e+05,5.367615e+05,5.475912e+05
676,K66,GO,4.950205e+05,4.497251e+05,4.392566e+05,4.962739e+05,5.733918e+05,6.272387e+05,7.331865e+05,8.506077e+05,7.858527e+05,7.192134e+05,7.407447e+05,7.549548e+05,7.503707e+05,7.922159e+05,8.407847e+05
292,C28,GO,1.134274e+06,1.080493e+06,1.086031e+06,1.249481e+06,1.472843e+06,1.631344e+06,1.848402e+06,2.203404e+06,2.525278e+06,2.160167e+06,2.378635e+06,2.862805e+06,2.868575e+06,2.973011e+06,3.055924e+06


## PPP Ratio

In [123]:
ppp_ratio = ppp2.copy()
ppp_ratio.loc[:,year_range] =  ppp2.loc[:,year_range] / ppp1.loc[:,year_range]
ppp_ratio.sample(5)

year,code,variable,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
147,C19,EMPE,1.990342,2.051869,2.043437,1.864261,1.563036,1.261782,1.146437,1.104161,0.936316,1.205031,1.0,0.817662,0.770752,0.833133,0.888484
247,C25,H_EMPE,1.323689,1.282841,1.262916,1.238987,1.189951,1.129391,1.102825,1.060414,0.996531,1.004372,1.0,0.973274,0.955466,0.940535,0.931745
423,F,H_EMPE,1.592469,1.460286,1.375576,1.315851,1.243436,1.168059,1.102143,1.057895,1.034439,1.023840,1.0,0.966492,0.925854,0.901660,0.883753
103,C16,H_EMPE,1.254346,1.214987,1.190173,1.148643,1.087725,1.077432,1.071976,1.038855,1.013618,1.035864,1.0,0.980954,0.948641,0.915297,0.890854
750,M72,VA_PI,1.201625,1.188373,1.154628,1.158256,1.182533,1.146844,1.107612,1.053702,1.012714,1.022194,1.0,0.979320,0.954953,0.921282,0.896141


## Load PPP Exchange Rate


$V_{GDP} / PPP$

In [138]:
PPP = pd.read_csv('ppp.csv')
PPP = pd.melt(PPP, id_vars=['ISO code','Country', 'Variable code', 'Variable name'], value_vars=[str(i) for i in year_range], var_name='year')
PPP.head(3)

,ISO code,Country,Variable code,Variable name,year,value
0,ABW,Aruba,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,4519.512695
1,AGO,Angola,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,37817.11719
2,AIA,Anguilla,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,273.9180298


In [137]:
NA_data = pd.read_excel('NA_data.xlsx', sheet_name='Data')
NA_data = NA_data[NA_data['year'].isin(year_range)][['countrycode', 'year', 'v_gdp']]
NA_data.sample(3)

,countrycode,year,v_gdp
5092,HUN,2004,2.107746e+07
4937,HND,2009,2.756322e+05
4270,GHA,2013,1.236500e+05


0       4.519513e+03
1       3.781712e+04
2       2.739180e+02
3       1.594262e+04
4       3.570514e+05
            ...     
2860             NaN
2861    7.053126e+06
2862    1.246796e+08
2863             NaN
2864    5.657001e-02
Name: value, Length: 2865, dtype: float64

In [150]:
PPP_exchange = pd.merge(left=PPP,
         right=NA_data,
         how='left',
         left_on=PPP['ISO code']+PPP['year'].astype(str),
         right_on=NA_data['countrycode']+NA_data['year'].astype(str),
         suffixes=('', '_y')
)
PPP_exchange['value'] = PPP_exchange['value'].str.replace(',','').astype(float)
PPP_exchange['value'] = PPP_exchange['v_gdp']/PPP_exchange['value']

PPP_exchange = PPP_exchange.drop(['key_0', 'countrycode','year_y','v_gdp'], axis=1)
PPP_exchange.dropna(inplace=True)
PPP_exchange

,ISO code,Country,Variable code,Variable name,year,value
0,ABW,Aruba,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,0.777543
1,AGO,Angola,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,3.241085
2,AIA,Anguilla,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,1.480888
3,ALB,Albania,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,31.437671
4,ARE,United Arab Emirates,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,1.087202
...,...,...,...,...,...,...
2852,VNM,Viet Nam,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2014,7333.596077
2853,YEM,Yemen,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2014,76.196263
2854,ZAF,South Africa,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2014,5.441281
2855,ZMB,Zambia,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2014,3.118285


## World Input Output Analysis

In [ ]:
# loading the temp.zip and creating a zip object
with ZipFile("wiot.zip", 'r') as zObject:
    zObject.extractall()

In [ ]:
%%time
df = pd.read_excel('WIOT2000_Nov16_ROW.xlsb', engine='pyxlsb')

CPU times: user 1min 8s, sys: 1.77 s, total: 1min 10s
Wall time: 1min 12s


In [ ]:
df.head(10)

,Intercountry Input-Output Table,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 2679,Unnamed: 2680,Unnamed: 2681,Unnamed: 2682,Unnamed: 2683,Unnamed: 2684,Unnamed: 2685,Unnamed: 2686,Unnamed: 2687,Unnamed: 2688
0,"43 countries, in current prices",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,(industry-by-industry),NaN,NaN,NaN,A01,A02,A03,B,C10-C12,C13-C15,...,CONS_np,CONS_g,GFCF,INVEN,CONS_h,CONS_np,CONS_g,GFCF,INVEN,GO
2,(millions of US$),NaN,NaN,NaN,"Crop and animal production, hunting and relate...",Forestry and logging,Fishing and aquaculture,Mining and quarrying,"Manufacture of food products, beverages and to...","Manufacture of textiles, wearing apparel and l...",...,Final consumption expenditure by non-profit or...,Final consumption expenditure by government,Gross fixed capital formation,Changes in inventories and valuables,Final consumption expenditure by households,Final consumption expenditure by non-profit or...,Final consumption expenditure by government,Gross fixed capital formation,Changes in inventories and valuables,Total output
3,NaN,NaN,NaN,NaN,AUS,AUS,AUS,AUS,AUS,AUS,...,USA,USA,USA,USA,ROW,ROW,ROW,ROW,ROW,TOT
4,NaN,NaN,NaN,NaN,c1,c2,c3,c4,c5,c6,...,c58,c59,c60,c61,c57,c58,c59,c60,c61,c62
5,A01,"Crop and animal production, hunting and relate...",AUS,r1,5876.833181,187.067919,65.079122,24.45264,9979.470649,249.425583,...,0,0,0.603338,0.031894,336.975473,0.000317,4.333551,8.914831,-0.000221,28947.959651
6,A02,Forestry and logging,AUS,r2,78.053348,41.448143,0.946121,3.572607,0.780698,0.097985,...,0,0,0.000202,0.000011,0.271302,0.000001,0.000018,0.086613,-0.000181,1506.081601
7,A03,Fishing and aquaculture,AUS,r3,10.916019,0.01848,8.363733,0.623371,254.706044,19.667171,...,0,0,0.000118,0.018981,163.504977,0.000126,0.001033,0.2445,0.000071,1512.383198
8,B,Mining and quarrying,AUS,r4,14.206129,0.243553,0.211697,996.265557,38.892554,2.423638,...,0,0,45.231545,0.003082,12.060419,0.000082,0.069017,16.497247,-0.038965,34012.527629
9,C10-C12,"Manufacture of food products, beverages and to...",AUS,r5,820.496849,7.001094,45.912387,45.81985,4809.581899,159.040816,...,0,0,1.962467,0.099112,1793.83206,0.000762,9.407714,0.923524,-0.000129,34168.387578


In [ ]:
# industries = df.iloc[5:, :3].reset_index(drop=True).copy()
# industries.columns = ['code', 'industry', 'country']
# print('Total number of industries',industries.shape[0])
# industries

In [ ]:
# value = df.iloc[5:, 4:industries.shape[0]+4].copy()
# value.columns = industries['code']
# value['industry'] = industries['code']
# value['country'] = industries['country']
# value.sample(5)

In [ ]:
# isProductive = ['K64', 'K65', 'K66', 'L68', 'M69_M70', 'N', 'O84', 'T', 'U']

# value[value['industry'].isin(isProductive)].groupby('country').sum().iloc[:,:-1].sum(axis=1)